In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/MC2.arff')
df = pd.DataFrame(data[0])

In [2]:
df.shape

(125, 40)

In [3]:
df.head()

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,Defective
0,16.0,19.0,0.0,0.0,22.0,30.0,10.0,0.20,12.0,2.50,...,23.0,0.11,70.0,121.0,11.0,13.0,90.0,30.56,50.0,b'N'
1,9.0,13.0,2.0,0.0,14.0,28.0,7.0,0.23,14.0,2.00,...,23.0,0.13,44.0,55.0,23.0,19.0,55.0,31.82,30.0,b'N'
2,2.0,3.0,0.0,0.0,0.0,4.0,2.0,0.25,2.0,2.00,...,7.0,0.17,17.0,17.0,7.0,6.0,12.0,0.00,8.0,b'N'
3,35.0,97.0,3.0,0.0,51.0,168.0,49.0,0.18,72.0,2.33,...,163.0,0.14,435.0,683.0,45.0,26.0,353.0,16.14,265.0,b'Y'
4,1.0,3.0,1.0,1.0,0.0,4.0,2.0,0.13,2.0,2.00,...,7.0,0.11,12.0,19.0,8.0,10.0,18.0,6.67,15.0,b'N'


In [4]:
buggy = b'Y'
clean = b'N'
x = df.drop(['Defective'],axis = 'columns')

for i in range(125):
    if df.iloc[i,39] == buggy:
        df.iloc[i,39] = 1
    else:
        df.iloc[i,39] = 0

y = df['Defective']
y = y.astype(int)
y.head()

0    0
1    0
2    0
3    1
4    0
Name: Defective, dtype: int32

In [5]:
buggy = 1
clean = 0

In [6]:
from sklearn.cluster import AffinityPropagation

af_model = AffinityPropagation(damping=0.9)
af_model.fit(x)
af_prediction = af_model.predict(x)

af_prediction

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1,
       5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [7]:
clusters = unique(af_prediction)
print(clusters)

[0 1 2 3 4 5]


In [8]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if af_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if af_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1, 0, 1, 1, 0, 0]


In [9]:
TP = 1
TN = 1
FP = 1
FN = 1

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == af_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient
MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.1858883730458876
Accuracy : 0.6666666666666666
F-Score : 0.1568627450980392


In [10]:
from sklearn.cluster import OPTICS

optics_model = OPTICS()
optics_prediction = optics_model.fit_predict(x)

optics_prediction[:5]

array([-1,  1,  4, -1,  4])

In [11]:
clusters = unique(optics_prediction)
print(clusters)

[-1  0  1  2  3  4  5  6  7  8]


In [12]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if optics_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if optics_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1]


In [13]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == optics_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.07613585928761199
Accuracy : 0.496
F-Score : 0.4878048780487805


In [14]:
from sklearn.cluster import MeanShift

MeanShift_model = MeanShift()
MeanShift_prediction = MeanShift_model.fit_predict(x)

clusters = unique(MeanShift_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if MeanShift_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if MeanShift_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 1
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == MeanShift_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.2429328990536748
Accuracy : 0.6746031746031746
F-Score : 0.163265306122449


In [15]:
from sklearn.cluster import DBSCAN

DBSCAN_model = DBSCAN()
DBSCAN_prediction = DBSCAN_model.fit_predict(x)

clusters = unique(DBSCAN_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if DBSCAN_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if DBSCAN_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == DBSCAN_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1

TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.06551856932831537
Accuracy : 0.35714285714285715
F-Score : 0.5207100591715976


In [16]:
from pyclustering.cluster.xmeans import xmeans

xmeans_instance = xmeans(x)
xmeans_instance.process()
groups = xmeans_instance.get_clusters()

xmeans_prediction = [0]*len(y)

for i in range(len(groups)):
    for u in groups[i]:
        xmeans_prediction[u] = i

clusters = unique(xmeans_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if xmeans_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if xmeans_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == xmeans_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.2863485028724909
Accuracy : 0.696
F-Score : 0.26923076923076916
